# Initialize data

In [71]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import librosa

bearingA = pd.read_excel("lozyska polaczone.xlsx",sheet_name='Łożysko_A').iloc[:,0:6].set_index('Time [s]')
bearingB = pd.read_excel("lozyska polaczone.xlsx",sheet_name='Łożysko_A').iloc[:,0:6].set_index('Time [s]')
#bearing_merged = pd.concat([bearingA, bearingB, bearingC, bearingD, bearingE, bearingF], axis = 1).set_index('Time [s]') 
bearingA, bearingB

(              A_P1       A_P2      A_P3      A_P4      A_P5
 Time [s]                                                   
 0.20004  -0.519396  -0.857482 -3.493313 -4.067562 -4.106657
 0.20016   2.180017  -2.306301 -4.304790 -4.888874 -7.063714
 0.20028   3.852659  -6.568164 -1.653053  2.848104 -8.963711
 0.20040  -1.038305  -9.880408 -3.249150  3.476073 -5.875597
 0.20052   1.854455  -6.060931 -6.129766 -2.603546 -1.718865
 ...            ...        ...       ...       ...       ...
 7.84668   5.163884  -3.730574 -3.578094  1.375764 -2.592770
 7.84680   5.414033  -0.391876 -5.562483  3.157149 -0.717225
 7.84692   2.640328 -11.806190 -4.744551  6.137130 -4.663985
 7.84704   3.450035 -13.799896 -1.833365  3.692787 -6.056617
 7.84716   4.667976  -3.848692  0.140911  0.377114 -4.198191
 
 [63727 rows x 5 columns],
               A_P1       A_P2      A_P3      A_P4      A_P5
 Time [s]                                                   
 0.20004  -0.519396  -0.857482 -3.493313 -4.067562 -4.10

In [40]:
bearing_merged #showing database

,A_P1,B_P1,C_P1,D_P1,E_P1,F_P1
Time [s],,,,,,
0.20004,-0.519396,22.390885,-183.841632,3.413330,1.882842,16.960818
0.20016,2.180017,-0.247097,21.977959,7.624334,-25.785791,112.672231
0.20028,3.852659,-22.271686,158.780062,0.791314,5.138907,59.805599
0.20040,-1.038305,4.636002,122.776109,-11.113140,27.986985,-46.282011
0.20052,1.854455,19.837660,-3.401899,-8.564543,9.005476,-102.990885
...,...,...,...,...,...,...
7.84668,5.163884,-21.751989,0.742538,17.251010,-5.661584,56.429173
7.84680,5.414033,54.031987,-6.154490,19.943821,23.741722,5.513519
7.84692,2.640328,28.846011,-4.571065,-11.800380,-19.209215,-75.359436


# Function creating features

Ekstrakcja cech jest kluczowym etapem w analizie dźwięku. Używamy biblioteki librosa do wydobycia różnych cech z nagrania dźwiękowego. W tym przypadku, ekstrahujemy następujące cechy:

**MFCCs (Mel-frequency cepstral coefficients)**: Są to współczynniki reprezentujące krótkoczasowe moc spektralną, które są szeroko stosowane w rozpoznawaniu mowy i analizie dźwięku. Dają one dobrą reprezentację kształtu spektralnego dźwięku.

**Chroma STFT (Short-time Fourier Transform)**: Odnosi się do całkowitej energii w każdym z dwunastu różnych pasmach chromatycznych, co daje informację o tonalności dźwięku.

**Spectral Contrast**: Opisuje różnicę amplitud między pikami a dolinami w spektrum dźwięku, co może wskazywać na różnorodność brzmień w nagraniu.

**Zero Crossing Rate**: Jest to częstotliwość, z jaką sygnał przechodzi przez zero. Jest to prosty wskaźnik charakterystyki dźwięku, który może być użyteczny w różnicowaniu dźwięków o różnej teksturze.

In [133]:
def features_creator(segment, sample_rate, condition):
    nyquist_freq = sample_rate / 2
    fmin = nyquist_freq / 8  
    n_bands = 2  
    spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sample_rate, fmin=fmin, n_bands=n_bands)
    mfccs = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sample_rate)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=segment)
    features = {        
        'condition': condition,
        'mfccs': np.mean(mfccs, axis=1),
        'chroma': np.mean(chroma_stft, axis=1),
        'spectral_contrast': np.mean(spectral_contrast, axis=1),
        'zero_crossing_rate': np.mean(zero_crossing_rate)}
    return features


In [131]:
def data_spliter_with_feature(db, sample_rate, segment_length, num_segment, condition):
    all_features = []
    for iter_seg in range(num_segment):
        start = int(iter_seg * sample_rate * segment_length)
        end = start + int(sample_rate * segment_length)
        segment = db[start:end]
        
        if len(segment) == int(sample_rate * segment_length):
            features = features_creator(segment, sample_rate, condition)
            if features:
                print(f"Segment {iter_seg}: Feature extraction successful")
            else:
                print(f"Segment {iter_seg}: Feature extraction failed")
        all_features.append(features)
    return 

In [132]:
sample_rate = 8335  # based on calculation
segment_length = 1  # length of each segment in seconds
num_segment = 10
bearingA_to_npArray = bearingA.iloc[:,1].to_numpy()
data_spliter_with_feature(bearingA_to_npArray,sample_rate, segment_length, num_segment, 'good')

Segment 0: Feature extraction successful
Segment 1: Feature extraction successful
Segment 2: Feature extraction successful
Segment 3: Feature extraction successful
Segment 4: Feature extraction successful
Segment 5: Feature extraction successful
Segment 6: Feature extraction successful
